# Geometry and Mesh for Tank 

To do: 
1. generate quadrilateral mesh more adapted to the flow; 
1. revolve mesh around axis to create a wedgew mesh; 

In [7]:
# This script generates a simple rectangular mesh using Gmsh
# and visualizes it directly through the Gmsh GUI.

# Try to load Gmsh.jl, handling potential naming differences
try
    using Gmsh: gmsh
catch
    using gmsh # Fallback for older versions or different setups
end

In [8]:
#..1/7: initialize gmsh
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1) # Set terminal output verbosity
gmsh.model.add("long_rectangular_mesh") # Name the CAD model

#..2/7: generate geometry parameters
h = 0.01 # Characteristic length for mesh elements
# Adjusted dimensions for a long rectangle, based on tank and nozzle example
Lx = 1.0 # Length of the rectangle in x-direction (corresponds to 'L' in tank example)
Ly = 0.1 # Height of the rectangle in y-direction (corresponds to 'R' in tank example)

#..define corner points of the rectangle via (x,y,z) coordinates
p1 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h, 1) # Bottom-left
p2 = gmsh.model.geo.addPoint(Lx,  0.0, 0.0, h, 2) # Bottom-right
p3 = gmsh.model.geo.addPoint(Lx,  Ly,  0.0, h, 3) # Top-right
p4 = gmsh.model.geo.addPoint(0.0, Ly,  0.0, h, 4) # Top-left

#..define lines (edges) connecting the points to form the sides
l1 = gmsh.model.geo.addLine(p1, p2, 1) # Bottom side
l2 = gmsh.model.geo.addLine(p2, p3, 2) # Right side
l3 = gmsh.model.geo.addLine(p3, p4, 3) # Top side
l4 = gmsh.model.geo.addLine(p4, p1, 4) # Left side

#..create the closed curve loop from the lines
curve_loop = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4], 1)

#..define the plane surface using the curve loop
plane_surface = gmsh.model.geo.addPlaneSurface([curve_loop], 1)

#..3/7: synchronize the CAD model
gmsh.model.geo.synchronize()

#..4/7: assign physical groups
# These groups are useful for applying boundary conditions in simulations
gmsh.model.addPhysicalGroup(1, [l1], -1, "Gamma_bottom")
gmsh.model.addPhysicalGroup(1, [l2], -1, "Gamma_right")
gmsh.model.addPhysicalGroup(1, [l3], -1, "Gamma_top")
gmsh.model.addPhysicalGroup(1, [l4], -1, "Gamma_left") # e.g., could be an inlet
gmsh.model.addPhysicalGroup(2, [plane_surface], -1, "Domain") # The main domain

#..5/7: generate two-dimensional mesh
# Set mesh algorithm and recombination options for quadrilaterals
gmsh.option.setNumber("Mesh.Algorithm", 8) # Netgen algorithm, often good for quads
gmsh.option.setNumber("Mesh.RecombineAll", 1) # Tell Gmsh to recombine triangles into quads
gmsh.model.geo.mesh.setRecombine(2, plane_surface) # Apply recombination specifically to this surface

gmsh.model.mesh.generate(2) # Generate 2D mesh

#..6/7: write mesh to file and visualize the mesh
mesh_filename = "long_rectangular_mesh.msh"
gmsh.write(mesh_filename) # Save the mesh to a .msh file
println("Mesh saved to: $mesh_filename")

gmsh.fltk.run() # Open the Gmsh GUI to display the generated mesh

#..7/7: finalize gmsh
gmsh.finalize()


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000770092s, CPU 0s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : Blossom: 2890 internal 220 closed
Info    : Blossom recombination completed (Wall 0.049758s, CPU 0.0625s): 1000 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.999918, min Q = 0.989606
Info    : Done meshing 2D (Wall 0.144869s, CPU 0.140625s)
Info    : 1111 nodes 1224 elements
Info    : Writing 'long_rectangular_mesh.msh'...
Info    : Done writing 'long_rectangular_mesh.msh'
Mesh saved to: long_rectangular_mesh.msh
-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Windows64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Buil